In [20]:
import pandas as pd 
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
ingr_ns=pd.read_csv('tabellen/data_ingr.csv')
ingr_schoon=pd.read_csv('tabellen/data_ingr_clean.csv')
data_recepten = pd.read_csv('tabellen/data_recipes.csv')
recipes = pd.read_csv('tabellen/Recepten_kiezen_gescheiden.csv') # Onze lijst van recepten die we wel en niet lekker vinden
recipes_wel = recipes[:80] # DataFrame met alle recpeten die wij lekker vinden
recipes_niet = recipes[80:] # DataFrame met alle recepten die wij niet lekker vinden

cal = data_recepten[(data_recepten['calories']>= 600)&(data_recepten['calories']<= 875)].copy()
cal
cal_ingr = ingr_schoon[ingr_schoon['recipe'].isin(cal['title'])].copy()
cal_ingr.to_csv('cal_ingr.csv', index=False)

In [3]:
# import csv
# with open('tabellen/Recepten_kiezen_f.csv', newline='') as csvfile:
#     csvlezer = csv.reader(csvfile, delimiter=',', quotechar='"')
#     for x in csvlezer:
#         x = recipes
# recipes[117:123]

# recipes

In [4]:
onze_ns = ingr_ns[ingr_ns['recipe'].isin(recipes['title'])] # DataFrame van de ongeschoonde data met alle ingredienten van onze gekozen gerechten
onze_schoon = ingr_schoon[ingr_schoon['recipe'].isin(recipes['title'])] # DataFrame van de opgeschoonde data met alle ingredienten van onze gekozen gerechten


In [5]:
cosine1 = np.dot([1,1,1],[1,0,0])/(norm([1,1,1])*norm([1,0,0]))
#cosine1 = np.dot([1,1,1,1,1,1],[1,1,0,1,0,0])/(norm([1,1,1,1,1,1])*norm([1,1,0,1,0,0]))
cosine2 = np.dot([1,1,1],[1,1,0])/(norm([1,1,1])*norm([1,1,0]))
#cosine2 = np.dot([1,1,1,1,1,1],[0,0,1,0,0,0])/(norm([1,1,1,1,1,1])*norm([0,0,1,0,0,0]))
cosine3 = np.dot([1,1,1,1,1,1],[0,0,1,1,0,0])/(norm([1,1,1,1,1,1])*norm([0,0,1,1,0,0]))
print(cosine1)
print(cosine2)
print(cosine3)

0.5773502691896258
0.8164965809277259
0.5773502691896258


In [6]:
fav = ['melk','ei','water']
rec1 = ['water', 'appel']
rec2 = ['water','ei']
count_vectorizer = CountVectorizer()
fav_matrix = count_vectorizer.fit_transform(fav)
rec1_matrix = count_vectorizer.transform(rec1)
rec2_matrix = count_vectorizer.transform(rec2)
print('fav_matrix:\n',fav_matrix.toarray())
print('rec1_matrix:\n',rec1_matrix.toarray())
print('rec2_matrix:\n',rec2_matrix.toarray())
s = cosine_similarity(np.ones(np.size(rec1_matrix.toarray())).reshape(1,-1), rec1_matrix.reshape(1,-1))
t = cosine_similarity(np.ones(np.size(rec2_matrix.toarray())).reshape(1,-1), rec2_matrix.reshape(1,-1))
print('cosine_similarity met rec1: ', s[0][0])
print('cosine_similarity met rec2: ', t[0][0])

print('np.ones:\n',np.ones(np.size(rec1_matrix.toarray())).reshape(1,-1))
print('reshape(1,-1)\n', rec1_matrix.reshape(1,-1).toarray())

fav_matrix:
 [[0 1 0]
 [1 0 0]
 [0 0 1]]
rec1_matrix:
 [[0 0 1]
 [0 0 0]]
rec2_matrix:
 [[0 0 1]
 [1 0 0]]
cosine_similarity met rec1:  0.4082482904638631
cosine_similarity met rec2:  0.5773502691896258
np.ones:
 [[1. 1. 1. 1. 1. 1.]]
reshape(1,-1)
 [[0 0 1 0 0 0]]


In [7]:
ingr_schoon['recipe']
ingr_schoon['ingredient'].loc[ingr_schoon['recipe']=='Kruidnoten in marsepein']
# a=[ingr_schoon['ingredient'].loc[ingr_schoon['recipe']==recept] for recept in recipes_wel['title']]
fav_recepten = recipes_wel['title'].tolist()
#a=np.array(list(set(ingr_schoon['ingredient'].loc[ingr_schoon['recipe'].isin(fav_recepten)])))

fav=list(set(ingr_schoon['ingredient'].loc[ingr_schoon['recipe'].isin(fav_recepten)]))
count_vectorizer = CountVectorizer()
fav_matrix = count_vectorizer.fit_transform(fav)
# for recept in ingr_schoon['recipe']:
#     if recept not in fav_recepten:
#         print(recept)
titel_lijst = []
[titel_lijst.append(recept) for recept in ingr_schoon['recipe'] if recept not in fav_recepten and recept not in titel_lijst]
#titel_lijst = [i for n, i in enumerate(ingr_schoon['recipe']) if i not in fav_recepten and i not in ingr_schoon['recipe'].iloc[0:n]]
#titel_lijst
gelijkenis = []
for recept in titel_lijst:
    recept_lijst = list(ingr_schoon['ingredient'].loc[ingr_schoon['recipe']==recept])
    rec_matrix = count_vectorizer.transform(recept_lijst)
    cosinus = cosine_similarity(np.ones(np.size(rec_matrix.toarray())).reshape(1,-1), rec_matrix.reshape(1,-1))
    gelijkenis.append([cosinus[0][0],recept])
    
gelijkenis.sort(reverse=True)
gelijkenis[:20]


[[0.10336227882434038, 'Snelle Thaise viskoekjes met chilisaus'],
 [0.08006407690254358, "Volkorenpincho's"],
 [0.08006407690254358, "Marokkaanse' lamsbout"],
 [0.08006407690254358, 'Kerstballetjes met gesmolten kaas'],
 [0.08006407690254358, "Burrito'-schotel met knapperige tortillapuntjes"],
 [0.08006407690254358, 'Banaan met chocolade en rum'],
 [0.08006407690254358, 'Asperges koken'],
 [0.08006407690254357, 'Yoghurtmarinade met baharat'],
 [0.07734924681537618,
  'Traybake met kikkererwten, kipgehaktballetjes en garam masala'],
 [0.07665551758398331,
  'Mexicaanse maaltijdsoep van zoete aardappel met kidneybonen '],
 [0.07665551758398331, 'Carpaccio-wraps'],
 [0.07548513560963974, 'Farro met paprika-tomatensaus'],
 [0.07412493166611012,
  'Snelle noedelroerbak met groene currypasta en een eitje'],
 [0.07412493166611012, 'Aardappel-tonijnsalade met asperges'],
 [0.07308816827558576, 'Minigroenten met dragondressing'],
 [0.07308816827558574, 'Roodfruit-avocadosmoothie'],
 [0.07203648

In [8]:
aanbeveling = [titel[1] for titel in gelijkenis][:20]
aanbeveling

['Snelle Thaise viskoekjes met chilisaus',
 "Volkorenpincho's",
 "Marokkaanse' lamsbout",
 'Kerstballetjes met gesmolten kaas',
 "Burrito'-schotel met knapperige tortillapuntjes",
 'Banaan met chocolade en rum',
 'Asperges koken',
 'Yoghurtmarinade met baharat',
 'Traybake met kikkererwten, kipgehaktballetjes en garam masala',
 'Mexicaanse maaltijdsoep van zoete aardappel met kidneybonen ',
 'Carpaccio-wraps',
 'Farro met paprika-tomatensaus',
 'Snelle noedelroerbak met groene currypasta en een eitje',
 'Aardappel-tonijnsalade met asperges',
 'Minigroenten met dragondressing',
 'Roodfruit-avocadosmoothie',
 'Zoete-aardappelstamppot met prei, geitenkaas en walnoten',
 'Macaronischotel met broccoli',
 'Italiaanse couscous ',
 'Tortelloni met verse kruiden en Parmezaan']

In [22]:
class Recepten: 
    
    def __init__(self, data_ingr, schoongemaakt=False): 
        # Initialisatie van de klasse met het pad naar de data en of de data schoon is 
        self.data_ingr = data_ingr
        self.schoongemaakt = schoongemaakt
        self.fav_ingr = None
        self.titel_lijst = None
        
    def laad_ingr(self):
        # Methode die de ingrediëntentabel inlaadt
        self.tabel = pd.read_csv(self.data_ingr)
        
    def aanbeveling(self, fav_recepten):
        self.fav_ingr = list(set(self.tabel['ingredient'].loc[self.tabel['recipe'].isin(fav_recepten)])) # door set worden de duplicates weggehaald
        #self.fav_ingr = np.array({self.tabel['ingredient'][i] for i in range(0, len(self.tabel)) if self.tabel['recipe'][i] in fav_recepten})
        count_vectorizer = CountVectorizer()
        self.fav_matrix = count_vectorizer.fit_transform(self.fav_ingr)
        titel_lijst = []
        [titel_lijst.append(recept) for recept in self.tabel['recipe'] if recept not in fav_recepten and recept not in titel_lijst]
        gelijkenis = []
        for recept in titel_lijst:
            recept_lijst = list(self.tabel['ingredient'].loc[self.tabel['recipe']==recept])
            rec_matrix = count_vectorizer.transform(recept_lijst)
            cosinus = cosine_similarity(np.ones(np.size(rec_matrix.toarray())).reshape(1,-1), rec_matrix.reshape(1,-1))
            gelijkenis.append([cosinus[0][0],recept])
            
        gelijkenis.sort(reverse=True)
        self.aanbeveling = [titel[1] for titel in gelijkenis][:20]
        
        return self.aanbeveling
        
recepten_klasse = Recepten('cal_ingr.csv',schoongemaakt=True)
recepten_klasse.laad_ingr()

rec = recipes_wel[:15]
lijst = ["Mexicaanse taco's met vis en zoete aardappel", 'Spruitjesgratin met zoete aardappel', 'Thaise kokosnoedelsoep met garnalen', 
             'Spaghetti bolognese met geraspte kaas', "Poppy's pancakes", 'Pita met gekruid gehakt en paprika-tomaatsalade', 
             'Indiase kipspies met spicy pilavrijst', 'Wraps met tomaat en gehakt', 'Pasta pesto met kipstuckjes en tomatensalade', 
             'Bietenquiche met witte kaas en walnoten', 'Krokante zalm met bloemkool en notenrijst', 'Gnocchi in tomatensaus met groenten', 
             'Romige spaghetti met doperwten en zalm', 'Spaghetti bolognese met geraspte kaas', 'Pastasalade pesto met burrata ', 
             'Rotirol', 'Broodje carpaccio', 'Biefstuksalade met avocado-chimichurridressing', 
             'Gnocchi met homemade gehaktballetjes en champignons', 'Pittige gehaktballetjes met snelle bami']
fav_recepten = rec['title'].tolist()
aanbevolen = recepten_klasse.aanbeveling(lijst)
aanbevolen

['Cottage pie met doperwten-linzen-maismix',
 'Pizza met regenboog van groenten',
 'Gehaktschotel met roomkaaspuree',
 'Rijkgevulde groente-enchilada’s met mais &amp; geraspte kaas',
 'Zalm en croûte met kruiden-olijvenboter',
 'Plaatpizza met salami en tomaatjes',
 'Gnocchi met walnoten, witlof &amp; basilicum',
 'Quinoa met gegrilde avocado en granaatappel',
 'Vegan pasta pesto',
 'Tagliatelle met spinazie en kibbeling',
 'Noedelroerbak met garnalen, rode curry &amp; broccoli',
 'Aspergequiche',
 'Gevulde ciabatta met Coburgerham en mozzarella',
 'Volkorenpasta met zalm en tomaat uit de oven',
 'Hartige taart met zoete puntpaprika &amp; oude kaas ',
 'Rijstsalade met omeletreepjes en basilicumolie',
 'Goudbruine broc\xadco\xadli\xad-to\xadnijn\xadtaart',
 'Fusilli met tomaat, verse kruiden en Parmezaan',
 'Andijvietaart',
 'Pizza Hawaï']

In [10]:
ingr_schoon.loc[ingr_schoon['recipe'] == 'Snelle Thaise viskoekjes met chilisaus ']

,Unnamed: 0,recipe,ingredient,quantity,unit
31220,45595,Snelle Thaise viskoekjes met chilisaus,verse koriander,15.0,g
31221,45596,Snelle Thaise viskoekjes met chilisaus,rode peper,20.0,g
31222,45597,Snelle Thaise viskoekjes met chilisaus,verse gember,10.0,g
31223,45598,Snelle Thaise viskoekjes met chilisaus,verse haricots verts,150.0,g
31225,45600,Snelle Thaise viskoekjes met chilisaus,vissaus,15.0,g
31226,45601,Snelle Thaise viskoekjes met chilisaus,limoen,60.0,g
31227,45602,Snelle Thaise viskoekjes met chilisaus,tarwebloem,120.0,g
31228,45603,Snelle Thaise viskoekjes met chilisaus,arachideolie,230.0,g
31229,45604,Snelle Thaise viskoekjes met chilisaus,zoete chilisaus,100.0,g


In [ ]:
fav_ingr = list(set(ingr_schoon['ingredient'].loc[ingr_schoon['recipe'].isin(fav_recepten)])) 
'diepvries rauwe gepelde roze garnalen' in fav_ingr

In [ ]:
# recept_schoon = Recepten(ingr_schoon)

# print(recept_schoon.data_frame)

In [ ]:
# recept_ns = Recepten(ingr_ns)

# print(recept_ns.data_frame)

In [ ]:
recipes_schoon = recipes.dropna(subset=['title'])
recipes_schoon

In [ ]:
a= np.array([recipes_schoon['title']]) 
a 

In [ ]:
set_ingr= { ingr_schoon['ingredient'][i] for i in range(0, len(ingr_schoon)) if ingr_schoon['recipe'][i] in a}
set_ingr 